In [1]:
from gensim import corpora, models, similarities
import gensim

In [2]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

import gensim
class MyReviews():


    def __iter__(self):

        review_item_iterator = iter(parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
        #for review_item in parse("../../download/reviews_Cell_Phones_and_Accessories.json.gz"): 
        for it in range(100000):
            if it%10000==0:
                print("it="+str(it))
            review_item = next(review_item_iterator)
            #print(review_item)
        #for review_item in self.reviews_cursor:


            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText = gensim.utils.simple_preprocess(reviewText)
                  

            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            yield preprocess_reviewText

In [3]:
import gensim
def myReviews(catFilter = None, asin = None, concatenate = False):
    preprocess_reviewText = []
    review_item_iterator = iter(parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
    it = 0
    conc_count = 0
    for review_item in review_item_iterator:
        it += 1
    #for it in range(1000000):
        if it%100000==0:
            print("it="+str(it))

        reviewText = review_item["reviewText"]
        label = review_item["asin"] + "_" + review_item["reviewerID"] 
        # TODO: stemming dazu und stop words filtern
        preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)


        #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
        if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
            if concatenate==False:
                yield preprocess_reviewText_i
            else:
                conc_count += 1
                preprocess_reviewText.extend(preprocess_reviewText_i)
                print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        print("Return the final text with length: "+str(len(preprocess_reviewText)))
        print(preprocess_reviewText)
        yield preprocess_reviewText

In [4]:
import os
dictionaryFileName = '../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.dict'
if os.path.isfile(dictionaryFileName)==False:
    #dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    dictionary = corpora.Dictionary(myReviews())
    from six import iteritems
    once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
    dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
    dictionary.compactify()  # remove gaps in id sequence after words that were removed
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)

print(dictionary)

Dictionary(128699 unique tokens: ['the', 'case', 'pictured', 'is', 'soft']...)


In [5]:
def myCorpus(catFilter = None, asin = None, concatenate = False):
    for review in myReviews(catFilter,asin,concatenate):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [6]:
import os
tfidfFileName ='../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat_corpus.tfidf'
if os.path.isfile(tfidfFileName)==False:
    #tfidf = models.TfidfModel(myCorpus("Cell Phones"))
    tfidf = models.TfidfModel(myCorpus())
    tfidf.save(tfidfFileName)
else:
    tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

TfidfModel(num_docs=3397849, num_nnz=136334127)


In [7]:
import pickle

tfidf_apple_5s_FileName ='../../download/word2vec/apple_5s_review.tfidf'
if os.path.isfile(tfidf_apple_5s_FileName)==False:
    apple_5s_corpus = next(myCorpus(asin="B00F3J4B5S",concatenate=True))
    apple_5s_corpus_tfidf = tfidf[apple_5s_corpus]  
    pickle.dump(apple_5s_corpus_tfidf, open(tfidf_apple_5s_FileName, 'wb'))
else:
    apple_5s_corpus_tfidf = pickle.load( open( tfidf_apple_5s_FileName, "rb" ) )
    
    


In [13]:
tbl = [[dictionary.get(tokenid),tfidf_value] for (tokenid,tfidf_value) in apple_5s_corpus_tfidf if tfidf_value > 0.1]
print(tbl)

[['the', 0.12472951813189243], ['is', 0.14423453325196334], ['of', 0.11993645776814608], ['phone', 0.14573451166486764], ['in', 0.11343799447184465], ['to', 0.13359384935411], ['that', 0.1308369745328605], ['you', 0.15486596389187587], ['with', 0.11088330205090342], ['have', 0.10113051442640471], ['iphone', 0.2880242322275483], ['an', 0.11170951963300968], ['new', 0.11469176282219677], ['apple', 0.2676926528861916], ['unlocked', 0.1447493116924237]]


In [ ]:
print(tfidf)